In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np

v1 = torch.tensor(1)
v2 = torch.tensor([1,1])
v3= torch.zeros((3,3))
v4 = torch.randn(3,3,3)

print(v1,v2,v3,v4)



tensor(1) tensor([1, 1]) tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]) tensor([[[ 0.4339, -0.3548, -0.6696],
         [ 1.1069,  1.3572,  0.2884],
         [ 0.8564,  0.8015, -0.0619]],

        [[ 1.0078, -0.5576, -0.7682],
         [ 1.0746,  0.4816,  0.4846],
         [-0.7232,  0.3940, -0.1431]],

        [[-0.5815, -0.8931, -0.2455],
         [ 1.2169, -0.2926,  0.2306],
         [ 0.0406, -1.2956, -0.6956]]])


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

Simple linear regression using Autograd

In [3]:
a = torch.randn(1, dtype=torch.float).to(device)
b = torch.randn(1, dtype=torch.float).to(device)
# and THEN set them as requiring gradients...
a.requires_grad_()
b.requires_grad_()
print(a, b)

tensor([-0.9740], requires_grad=True) tensor([0.7368], requires_grad=True)


In [4]:
x_train = np.random.rand(100, 1)
y_train = 1 + 2 * x_train + .1 * np.random.randn(100, 1)
x_train_tensor = torch.from_numpy(x_train).float().to(device)
y_train_tensor = torch.from_numpy(y_train).float().to(device)

In [6]:
lr = 1e-1
n_epochs = 20
for epoch in range(n_epochs):
  yhat = a + b * x_train_tensor
  error = y_train_tensor - yhat
  loss = (error ** 2).mean()
  loss.backward()
  print(a.grad)
  print(b.grad)
  with torch.no_grad():
      a -= lr * a.grad
      b -= lr * b.grad

  a.grad.zero_()
  b.grad.zero_()
print(a, b)

tensor([-0.0083])
tensor([-0.0250])
tensor([-0.0043])
tensor([-0.0226])
tensor([-0.0013])
tensor([-0.0208])
tensor([0.0010])
tensor([-0.0193])
tensor([0.0026])
tensor([-0.0182])
tensor([0.0038])
tensor([-0.0173])
tensor([0.0047])
tensor([-0.0166])
tensor([0.0054])
tensor([-0.0160])
tensor([0.0058])
tensor([-0.0155])
tensor([0.0061])
tensor([-0.0150])
tensor([0.0063])
tensor([-0.0147])
tensor([0.0065])
tensor([-0.0143])
tensor([0.0065])
tensor([-0.0141])
tensor([0.0066])
tensor([-0.0138])
tensor([0.0066])
tensor([-0.0135])
tensor([0.0066])
tensor([-0.0133])
tensor([0.0065])
tensor([-0.0131])
tensor([0.0065])
tensor([-0.0129])
tensor([0.0064])
tensor([-0.0127])
tensor([0.0063])
tensor([-0.0125])
tensor([1.0580], requires_grad=True) tensor([1.8659], requires_grad=True)


Build a fully connected neural network

In [8]:
import torch
import torchvision
from torchvision import transforms, datasets

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Load MNIST data

In [9]:
train = torchvision.datasets.MNIST('', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))

test = torchvision.datasets.MNIST('', train=False, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))

trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)

testset = torch.utils.data.DataLoader(test, batch_size=10, shuffle=False)

Create your fully connected NN 

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1) 


In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device
net = Net().to(device)
print(net)

Net(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)


Define the Loss function and optimizors

In [12]:
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.005)

Training with autograd

In [15]:
for epoch in range(5): 
    for data in trainset:  
        X, y = data
        X = X.to(device)
        y = y.to(device) 
        break
        net.zero_grad()  
        output = net(X.view(-1,784))  
        loss = loss_criterion(output, y)  
        loss.backward()  
        optimizer.step()  
    print(loss)  

tensor(0.0467, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0467, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0467, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0467, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0467, device='cuda:0', grad_fn=<MeanBackward0>)


Evaluate the test accuracy

In [1]:
correct = 0
total = 0

with torch.no_grad():
    for data in testset:
        X, y = data
        X = X.to(device)
        y = y.to(device)
        break
        output = net(X.view(-1,784))
        
        for idx, i in enumerate(output):
          if torch.argmax(i) == y[idx]:
            correct += 1
          total += 1

print("Accuracy: ", round(correct/total, 2))

NameError: ignored